In [4]:
# import stuff
import pandas as p
import numpy as n
import seaborn as s

## Data Set 2; Stock Exchange Data

This is the data set containing the stock market information for 14 unique exchanges over the span of 12-31-1965 to 06-03-2021.

There are 3 csv files, but for this project, we will only be using indexData.csv. This has all the null values and extra data that indexProcessed.csv does not. 

[csv link](https://www.kaggle.com/mattiuzc/stock-exchange-data)


### Useful Stats:
- Size of set ~> 112,458
- Columns ~> 8

In [6]:
stock = p.read_csv('data/indexData.csv', header = None, skipinitialspace = True, encoding = 'latin-1', low_memory = False)
len(stock)

112458